In [2]:
import pandas as pd
import numpy as np

In [9]:
# each row represent a side of the building, for instance, it could be the
# north side of the building, so this row would have the u value

facade_setting = pd.read_csv("house.csv", index_col=None)
facade_setting = facade_setting.drop(facade_setting.columns[0], axis=1)
# List of possible orientations

# orientations = ['north', 'east', 'south', 'west']
orientation_num = [1, 2, 3, 4]

# Add the new column 'orientation' with random values from the orientations list
facade_setting['orientation'] = np.random.choice(orientation_num, size=len(facade_setting))
facade_setting.head(10)

,U_value,SHGC_value,WWR_value,Window_Num,SC_value,Climate_Zone,Cluster,Energy_Rating,orientation
0,2.123353,0.234743,0.489698,11,0.496152,8,1,B,2
1,1.981524,0.325493,0.467486,17,0.569917,6,2,C,3
2,4.830145,0.633332,0.569683,4,0.268770,3,0,A,3
3,3.116825,0.730655,0.599659,16,0.778715,7,0,A,2
4,4.534847,0.416496,0.478571,17,0.277962,7,1,B,4
5,2.831032,0.211346,0.445446,8,0.897567,8,1,B,1
6,4.150219,0.239358,0.476536,12,0.585315,6,1,B,1
7,4.402815,0.643583,0.315479,16,0.354661,5,0,A,4
8,4.831142,0.617443,0.365526,17,0.594093,8,2,C,1
9,1.708513,0.605050,0.478905,8,0.167014,1,4,E,3


Energy demand assessment

In [10]:

def calculate_energy_demand(row):
    u_value = row['U_value']
    shgc_value = row['SHGC_value']
    wwr_value = row['WWR_value']
    window_num = row['Window_Num']
    sc_value = row['SC_value']
    climate_zone = row['Climate_Zone']
    orientation = row['orientation']
    
    # Simplified calculations for energy demand
    # more precise and complex calculating metrics could be used to 
    # assess the demand
    cooling_need = (u_value * shgc_value * wwr_value * window_num * sc_value) * (climate_zone / 10)
    heating_need = (u_value * wwr_value * window_num * (1 - sc_value)) * (climate_zone / 10)
    
    # Assume north orientation reduces lighting need, while south increases it
    orientation_factor = 1.0
    if orientation == 'north':
        orientation_factor = 0.8
    elif orientation == 'south':
        orientation_factor = 1.2
    
    lighting_need = (1 - u_value) * (1 - wwr_value) * window_num * sc_value * orientation_factor
    
    return pd.Series([cooling_need, heating_need, lighting_need])

# Apply the function to each row
facade_setting[['cooling_need', 'heating_need', 'lighting_need']] = facade_setting.apply(calculate_energy_demand, axis=1)

In [11]:
facade_setting.head()

,U_value,SHGC_value,WWR_value,Window_Num,SC_value,Climate_Zone,Cluster,Energy_Rating,orientation,cooling_need,heating_need,lighting_need
0,2.123353,0.234743,0.489698,11,0.496152,8,1,B,2,1.065716,4.610342,-3.128604
1,1.981524,0.325493,0.467486,17,0.569917,6,2,C,3,1.752756,4.063683,-5.063987
2,4.830145,0.633332,0.569683,4,0.268770,3,0,A,3,0.562065,2.414507,-1.771923
3,3.116825,0.730655,0.599659,16,0.778715,7,0,A,2,11.910397,4.632200,-10.558761
4,4.534847,0.416496,0.478571,17,0.277962,7,1,B,4,2.989867,18.647292,-8.709643


Model training and evaluation

In [14]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prepare the data
X = facade_setting.drop(columns=['Energy_Rating', 'Cluster'])
y = facade_setting['Energy_Rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc}')

Accuracy: 0.83


In [15]:
print(y_pred)

['B' 'C' 'B' 'C' 'E' 'B' 'B' 'A' 'D' 'A' 'B' 'B' 'C' 'B' 'A' 'E' 'E' 'A'
 'B' 'B' 'D' 'A' 'A' 'E' 'E' 'C' 'B' 'B' 'B' 'D' 'B' 'C' 'A' 'B' 'B' 'A'
 'C' 'D' 'B' 'C' 'A' 'A' 'A' 'C' 'E' 'A' 'B' 'C' 'A' 'A' 'E' 'B' 'D' 'E'
 'C' 'B' 'D' 'A' 'C' 'B' 'C' 'B' 'B' 'A' 'D' 'C' 'D' 'C' 'E' 'C' 'E' 'C'
 'C' 'A' 'E' 'D' 'D' 'A' 'A' 'D' 'C' 'E' 'A' 'D' 'D' 'C' 'A' 'E' 'D' 'D'
 'E' 'B' 'B' 'C' 'A' 'D' 'D' 'B' 'B' 'C' 'E' 'E' 'B' 'E' 'B' 'B' 'B' 'D'
 'A' 'A' 'C' 'A' 'C' 'E' 'C' 'B' 'A' 'D' 'E' 'A' 'A' 'D' 'B' 'D' 'D' 'B'
 'C' 'A' 'D' 'C' 'C' 'B' 'D' 'E' 'C' 'B' 'C' 'C' 'B' 'C' 'D' 'D' 'D' 'D'
 'B' 'A' 'A' 'D' 'D' 'C' 'C' 'E' 'B' 'A' 'D' 'E' 'E' 'A' 'B' 'B' 'E' 'B'
 'D' 'D' 'D' 'A' 'A' 'E' 'E' 'C' 'C' 'C' 'A' 'D' 'E' 'B' 'B' 'E' 'A' 'D'
 'C' 'E' 'A' 'E' 'B' 'B' 'B' 'A' 'A' 'E' 'E' 'B' 'B' 'C' 'E' 'E' 'A' 'A'
 'D' 'E']


In [16]:
# only ratings A and B are considered "energy efficient facade setting"
threshold_rate = "B"

In [59]:
from sklearn.metrics.pairwise import euclidean_distances

def pipeline(facade_setting, model, user_input):
    # Step 1: Calculate energy demand
    energy_demand = calculate_energy_demand(user_input)
    energy_demand.index = ['cooling_need', 'heating_need', 'lighting_need']
    user_input = pd.concat([user_input, energy_demand])
    
    # Step 2: Predict energy rating
    X_user = user_input.drop(['Energy_Rating', 'Cluster'], errors='ignore')
    X_user = X_user.to_frame().T
    predicted_rating = model.predict(X_user)[0]
    
    # Step 3: Check energy rating
    if predicted_rating in ['A', 'B']:
        return "The setting given is okay."
    else:
        # Step 4: Find similar rows
        X_train = facade_setting.drop(columns=['Energy_Rating', 'Cluster'])
        y_train = facade_setting['Energy_Rating']
        
        # Filter training set for rows with 'A' or 'B' rating
        facade_setting_ab = facade_setting[y_train.isin(['A', 'B'])]
        X_train_ab = X_train[y_train.isin(['A', 'B'])]
        
        # Calculate distances
        distances = euclidean_distances(X_train_ab, X_user)
        closest_indices = np.argsort(distances, axis=0)[:3].flatten()
        
        # Get the top 3 most similar rows
        similar_rows = facade_setting_ab.iloc[closest_indices]
        # test = similar_rows.iloc[0]
        # test = test.drop(["Energy_Rating", "Cluster"])
        # print(model.predict(test.to_frame().T))
        
        # Format the results
        results = []
        orientations = {1: 'north', 2: 'east', 3: 'south', 4: 'west'}
        for i, row in similar_rows.iterrows():
            orientation = orientations[row['orientation']]
            result = f"Solution {len(results) + 1}: for {orientation} side of the building (possible parameters): "
            result += f"u_value: {row['U_value']:.3f}, "
            result += f"shgc_value: {row['SHGC_value']:.3f}, "
            result += f"wwr_value: {row['WWR_value']:.3f}, "
            result += f"window_num: {row['Window_Num']}, "
            result += f"sc_value: {row['SC_value']:.3f}, "
            result += f"Rating: {row['Energy_Rating']}"
            results.append(result)
        
        return "\n".join(results)

In [60]:
# Example user input
user_input = pd.Series({
    'U_value': 2.0,
    'SHGC_value': 0.3,
    'WWR_value': 0.5,
    'Window_Num': 10,
    'SC_value': 0.4,
    'Climate_Zone': 5,
    'orientation': 1  # Assuming orientation is represented as a number
})

# Run the pipeline
result = pipeline(facade_setting, model, user_input)
print(result)

Solution 1: for north side of the building (possible parameters): u_value: 1.790, shgc_value: 0.354, wwr_value: 0.496, window_num: 9, sc_value: 0.519, Rating: B
Solution 2: for south side of the building (possible parameters): u_value: 2.762, shgc_value: 0.656, wwr_value: 0.516, window_num: 9, sc_value: 0.415, Rating: A
Solution 3: for east side of the building (possible parameters): u_value: 1.678, shgc_value: 0.375, wwr_value: 0.489, window_num: 11, sc_value: 0.277, Rating: B
